In [ ]:
pip install backtrader

In [ ]:
import backtrader as bt
import pandas as pd
import yfinance as yf

class OBVWindow(bt.Indicator):
    lines = ('obv_window',)
    params = (('window', 15),)

    def __init__(self):
        self.addminperiod(self.params.window)

    def next(self):
        # 롤링 OBV 초기화
        rolling_obv = 0

        # 롤링 윈도우에 대해 OBV 변경 사항 계산
        for i in range(1, self.params.window + 1):
            obv_change = 0
            if self.data.close[-i] > self.data.close[-i - 1]:  # 가격 상승
                obv_change = self.data.volume[-i]
            elif self.data.close[-i] < self.data.close[-i - 1]:  # 가격 하락
                obv_change = -self.data.volume[-i]

            # 롤링 OBV에 누적
            rolling_obv += obv_change

        # 계산된 롤링 OBV를 라인에 할당
        self.lines.obv_window[0] = rolling_obv



# Trading strategy using OBV and CCI
class OBVStrategy(bt.Strategy):
    params = (
        ('obv_window', 10),
        ('cci_period', 18),
    )

    def __init__(self):
        # Add indicators
        self.obv = OBVWindow(self.data, window=self.params.obv_window)
        self.rsi = bt.indicators.RSI(self.data.close)
        self.cci = bt.indicators.CommodityChannelIndex(
            self.data, period=self.params.cci_period
        )
        self.buy_price = 0

        # Initialize lists to record buy and sell dates
        self.buy_dates = []  # 매수 날짜 저장
        self.sell_dates = []  # 매도 날짜 저장

    def next(self):

        date = self.data.datetime.date(0)  # 현재 데이터의 날짜
        cash = self.broker.get_cash()      # 현재 가용 자금
        price = self.data.close[0]         # 현재 종가
        size = int(cash / price)           # 살 수 있는 최대 주식 수
        #print(f"{self.data.datetime.date(0)} {price}, have a cash: {cash}, position.size : {self.position.size}")
        if self.position:  # If in position
            # Sell if OBV is below 0
            if self.obv[0] < 0:
                self.sell(size=self.position.size)
                #print(f"SELL: {self.data.close[0]:.2f} on {self.data.datetime.date(0)}, buy : {self.buy_price}, sell : {price}, have a cash: {cash}")
                self.sell_dates.append(str(self.data.datetime.date(0)))  # 매도 날짜 기록
        else:  # If not in position
            # Buy if OBV is above 0
            if self.obv[0] > 0 and size > 0:
                self.buy_price = self.data.close[0]
                self.buy(size=size)
                #print(f"BUY: {self.data.close[0]:.2f} on {self.data.datetime.date(0)} position :{self.position.size}, have a cash: {cash}")
                self.buy_dates.append(str(self.data.datetime.date(0)))  # 매수 날짜 기록


# Backtesting function for multiple stocks
def backtest_stocks(stock_list, start_date, end_date):
    results = []

    for stock in stock_list:
        print(f"\nBacktesting for {stock}...")

        # Fetch data using yfinance
        data = yf.download(stock, start=start_date, end=end_date)

        # Validate data
        if data.empty:
            print(f"No data for {stock}. Skipping...")
            continue

        # Rename columns to match Backtrader's format
        try:
            data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
            data.columns = ['open', 'high', 'low', 'close', 'volume']
        except KeyError as e:
            #print(f"Error: Missing columns in data for {stock}. Skipping... ({e})")
            continue

        # Convert to Backtrader data feed
        try:
            data_feed = bt.feeds.PandasData(dataname=data)
        except Exception as e:
            #print(f"Error converting data for {stock} to Backtrader format: {e}")
            continue

        # Set up Backtrader engine
        cerebro = bt.Cerebro()
        cerebro.adddata(data_feed)
        cerebro.addstrategy(OBVStrategy)
        cerebro.broker.set_cash(1000)  # Initial capital
        cerebro.broker.setcommission(commission=0.001)  # Commission rate

        # Run strategy
        strategies = cerebro.run()  # Run returns a list of strategy instances
        strategy_instance = strategies[0]  # 첫 번째 전략 인스턴스 가져오기

        # Collect buy and sell dates from the strategy
        buy_dates = strategy_instance.buy_dates  # 첫 번째 전략의 buy_dates 리스트
        sell_dates = strategy_instance.sell_dates  # 첫 번째 전략의 sell_dates 리스트

        # Calculate performance metrics
        try:
            start_value = cerebro.broker.startingcash
            end_value = cerebro.broker.getvalue()
            buy_and_hold_profit = data['close'].iloc[-1] / data['close'].iloc[0] * start_value
            strategy_return = (end_value - start_value) / start_value * 100
            buy_and_hold_return = (buy_and_hold_profit - start_value) / start_value * 100
        except Exception as e:
            #print(f"Error calculating performance metrics for {stock}: {e}")
            continue

        # Append results
        results.append({
            'Stock': stock,
            'Strategy Return (%)': strategy_return,
            'Buy-and-Hold Return (%)': buy_and_hold_return
        })

        # Print individual stock results
        print()
        print(f"Strategy Return: {strategy_return:.2f}%")
        print(f"Buy-and-Hold Return: {buy_and_hold_return:.2f}%")

    # Return results as a DataFrame
    return pd.DataFrame(results), buy_dates, sell_dates

# Example usage
if __name__ == "__main__":
    stock_list = [
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "AMZN",  # Amazon
    "GOOGL", # Google (Alphabet Inc.)
    "META",  # Meta
    "TSLA",  # Tesla
    "NVDA",  # NVIDIA
    "NFLX",  # Netflix
    "ADBE",  # Adobe
    "INTC"   # Intel
]
    start_date = '2023-01-01'
    end_date = '2024-12-31'

    results_df, buy_dates, sell_dates= backtest_stocks(stock_list, start_date, end_date)
    print("\nSummary of Results:")
    print(results_df)


[*********************100%***********************]  1 of 1 completed


Backtesting for AAPL...



[*********************100%***********************]  1 of 1 completed


Strategy Return: 22.51%
Buy-and-Hold Return: 106.44%

Backtesting for MSFT...



[*********************100%***********************]  1 of 1 completed

...


[*********************100%***********************]  1 of 1 completed


Strategy Return: 35.14%
Buy-and-Hold Return: 216.03%

Backtesting for ADBE...



[*********************100%***********************]  1 of 1 completed


Strategy Return: -3.83%
Buy-and-Hold Return: 32.95%

Backtesting for INTC...



Strategy Return: -9.50%
Buy-and-Hold Return: -23.68%

Summary of Results:
   Stock  Strategy Return (%)  Buy-and-Hold Return (%)
0   AAPL            22.510396               106.444401
1   MSFT            31.909644                83.375066
2   AMZN            36.391008               166.895833
3  GOOGL            22.745643               120.051610
4   META            54.892421               387.213412
5   TSLA            41.731028               327.641078
6   NVDA            61.987998               879.531997
7   NFLX            35.141232               216.026430
8   ADBE            -3.832118                32.951438
9   INTC            -9.495368               -23.681257
